### Analyzing the Effects of Top/Bottom Coding on The Accuracy of Exponential Smoothing Forecasts

***

***

## Import Modules

Note the functions imported from `helper_functions`, as these are custom functions written by the paper authors. See `helper_functions.py` for comments and functions descriptions.

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer, Detrender

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import full_coding_analysis

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

Import data.

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
detrender = Detrender()
detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
Y = pd.concat(detrended_series, axis=1).T

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [6]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [7]:
results_dict_ses

{'h=1, Top 0.1': {'Mean Accuracies': array([0.3764, 0.3764]),
  'Protected Mean Accuracies:': array([0.3913, 0.3913]),
  '% Change Mean accuracy:': array([-0.0397, -0.0397]),
  '% Change Median accuracy:': array([-0.1483, -0.1483]),
  '% Forecasted Points adjusted downward:': 0.1524,
  '% Forecasted Points adjusted upward:': 0.8476,
  '% Series with improved accuracy:': array([0.4451, 0.4451]),
  '% Series with reduced accuracy:': array([0.5549, 0.5549]),
  'Original Mean Absolute Error Upward Adjusted:': 0.4266,
  'Original Mean Absolute Error Downward Adjusted:': 0.0977,
  'Protected Mean Absolute Error Upward Adjusted:': 0.429,
  'Protected Mean Absolute Error Downward Adjusted:': 0.1822},
 'h=20, Top 0.1': {'Mean Accuracies': array([0.4321, 0.5414]),
  'Protected Mean Accuracies:': array([0.4319, 0.5408]),
  '% Change Mean accuracy:': array([0.0006, 0.001 ]),
  '% Change Median accuracy:': array([-0.0479, -0.0159]),
  '% Forecasted Points adjusted downward:': 0.1768,
  '% Forecaste

In [8]:
original_forecasts = fcasts_ses['h=20, Bottom 0.1']
protected_forecasts = fcasts_protected_ses['h=20, Bottom 0.1']
test = tests['h=20, Bottom 0.1']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.4580970384373031

In [13]:
np.mean(adjusted_up[improved])

0.5419029615626969

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [16]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [17]:
results_dict_des

{'h=1, Top 0.1': {'Mean Accuracies': array([0.3765, 0.3765]),
  'Protected Mean Accuracies:': array([0.3924, 0.3924]),
  '% Change Mean accuracy:': array([-0.0422, -0.0422]),
  '% Change Median accuracy:': array([-0.1072, -0.1072]),
  '% Forecasted Points adjusted downward:': 0.1768,
  '% Forecasted Points adjusted upward:': 0.8232,
  '% Series with improved accuracy:': array([0.3963, 0.3963]),
  '% Series with reduced accuracy:': array([0.6037, 0.6037]),
  'Original Mean Absolute Error Upward Adjusted:': 0.406,
  'Original Mean Absolute Error Downward Adjusted:': 0.2392,
  'Protected Mean Absolute Error Upward Adjusted:': 0.4098,
  'Protected Mean Absolute Error Downward Adjusted:': 0.3115},
 'h=20, Top 0.1': {'Mean Accuracies': array([0.4919, 0.6053]),
  'Protected Mean Accuracies:': array([0.4959, 0.6081]),
  '% Change Mean accuracy:': array([-0.0081, -0.0046]),
  '% Change Median accuracy:': array([-0.0678, -0.0499]),
  '% Forecasted Points adjusted downward:': 0.2146,
  '% Forecas

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [20]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [21]:
results_dict_tes

{'h=1, Top 0.1': {'Mean Accuracies': array([0.4045, 0.4045]),
  'Protected Mean Accuracies:': array([0.4191, 0.4191]),
  '% Change Mean accuracy:': array([-0.0362, -0.0362]),
  '% Change Median accuracy:': array([-0.1354, -0.1354]),
  '% Forecasted Points adjusted downward:': 0.2683,
  '% Forecasted Points adjusted upward:': 0.7317,
  '% Series with improved accuracy:': array([0.4329, 0.4329]),
  '% Series with reduced accuracy:': array([0.5671, 0.5671]),
  'Original Mean Absolute Error Upward Adjusted:': 0.3974,
  'Original Mean Absolute Error Downward Adjusted:': 0.4238,
  'Protected Mean Absolute Error Upward Adjusted:': 0.3966,
  'Protected Mean Absolute Error Downward Adjusted:': 0.4805},
 'h=20, Top 0.1': {'Mean Accuracies': array([0.5688, 0.697 ]),
  'Protected Mean Accuracies:': array([0.5677, 0.6949]),
  '% Change Mean accuracy:': array([0.0019, 0.0031]),
  '% Change Median accuracy:': array([-0.0114,  0.0103]),
  '% Forecasted Points adjusted downward:': 0.3415,
  '% Forecast